## Work
1. 請比較使用不同層數以及不同 Dropout rate 對訓練的效果
2. 將 optimizer 改成使用 Adam 並加上適當的 dropout rate 檢視結果

In [27]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [28]:
train, test = keras.datasets.cifar10.load_data()

In [29]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [30]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [34]:
from keras.layers import Dropout
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128], drp_ratio=0.2):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = Dropout(drp_ratio)(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = Dropout(drp_ratio)(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [35]:
"""Code Here
設定超參數
"""
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
Dropout_EXP = [0.5, 0.25, 0.1]

In [36]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for dropout in Dropout_EXP:
    model = build_mlp(input_shape=x_train.shape[1:], drp_ratio=dropout)
    model.summary()
    optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)

    # Collect results    
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    name_tag = 'drop_rate_%.2f' % dropout
    results[name_tag] = {'train-loss': train_loss,
                         'valid-loss': valid_loss,
                         'train-acc': train_acc,
                         'valid-acc': valid_acc}


W0719 22:32:38.119325 30044 deprecation.py:506] From C:\Users\a2279\.conda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0719 22:32:38.314800 30044 deprecation_wrapper.py:119] From C:\Users\a2279\.conda\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

W0719 22:32:38.529227 30044 deprecation.py:323] From C:\Users\a2279\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 58s 1ms/step - loss: 2.2587 - acc: 0.1351 - val_loss: 2.0990 - val_acc: 0.1774
Epoch 2/50
50000/50000 [==============================] - 12s 240us/step - loss: 2.1566 - acc: 0.1589 - val_loss: 2.1265 - val_acc: 0.1872
Epoch 3/50
50000/50000 [==============================] - 12s 249us/step - loss: 2.1236 - acc: 0.1736 - val_loss: 2.0316 - val_acc: 0.2405
Epoch 4/50
50000/50000 [==============================] - 12s 231us/step - loss: 2.0979 - acc: 0.1905 - val_loss: 2.0430 - val_acc: 0.2389
Epoch 5/50
50000/50000 [==============================] - 12s 230us/step - loss: 2.0885 - acc: 0.1948 - val_loss: 2.0280 - val_acc: 0.2550
Epoch 6/50
50000/50000 [==============================] - 11s 229us/step - loss: 2.0779 - acc: 0.1985 - val_loss: 2.0040 - val_acc: 0.2470
Epoch 7/50
50000/50000 [==============================] - 11s 228us/step - loss: 2.0773 - acc: 0.1998 - val_loss: 2.02

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 11s 213us/step - loss: 2.0610 - acc: 0.2362 - val_loss: 1.8408 - val_acc: 0.3297
Epoch 2/50
50000/50000 [==============================] - 10s 199us/step - loss: 1.8924 - acc: 0.3137 - val_loss: 1.7943 - val_acc: 0.3634
Epoch 3/50
50000/50000 [==============================] - 10s 197us/step - loss: 1.8465 - acc: 0.3295 - val_loss: 1.7284 - val_acc: 0.3773
Epoch 4/50
50000/50000 [==============================] - 10s 200us/step - loss: 1.8065 - acc: 0.3441 - val_loss: 1.6897 - val_acc: 0.3977
Epoch 5/50
50000/50000 [==============================] - 10s 204us/step - loss: 1.7782 - acc: 0.3545 - val_loss: 1.6696 - val_acc: 0.4042
Epoch 6/50
50000/50000 [==============================] - 10s 202us/step - loss: 1.7543 - acc: 0.3619 - val_loss: 1.6603 - val_acc: 0.4096
Epoch 7/50
50000/50000 [==============================] - 10s 201us/step - loss: 1.7392 - acc: 0.3708 - val_loss: 1.

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 11s 224us/step - loss: 1.9882 - acc: 0.2709 - val_loss: 1.7803 - val_acc: 0.3529
Epoch 2/50
50000/50000 [==============================] - 11s 217us/step - loss: 1.7791 - acc: 0.3595 - val_loss: 1.7256 - val_acc: 0.3829
Epoch 3/50
50000/50000 [==============================] - 11s 210us/step - loss: 1.7033 - acc: 0.3884 - val_loss: 1.6369 - val_acc: 0.4103
Epoch 4/50
50000/50000 [==============================] - 12s 240us/step - loss: 1.6399 - acc: 0.4177 - val_loss: 1.5536 - val_acc: 0.4464
Epoch 5/50
50000/50000 [==============================] - 12s 231us/step - loss: 1.5994 - acc: 0.4290 - val_loss: 1.5382 - val_acc: 0.4530
Epoch 6/50
50000/50000 [==============================] - 11s 225us/step - loss: 1.5754 - acc: 0.4368 - val_loss: 1.5146 - val_acc: 0.4622
Epoch 7/50
50000/50000 [==============================] - 11s 214us/step - loss: 1.5448 - acc: 0.4463 - val_loss: 1.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
color_bar = ["r", "g", "b"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()